In [1]:
import mysql.connector
from dotenv import load_dotenv
from decouple import config 
import os

In [2]:
load_dotenv()

connection = mysql.connector.connect(
    host= os.getenv("host"),
    user= os.getenv("user"),
    password= os.getenv("password"),
    database="music_trends"
)

cursor = connection.cursor()

In [3]:
music_trends = """CREATE DATABASE "music_trends";"""

In [5]:
songs = """
CREATE TABLE IF NOT EXISTS songs (
    id INT AUTO_INCREMENT PRIMARY KEY,
    song VARCHAR(255) NOT NULL,
    artist VARCHAR(255) NOT NULL
); """

cursor.execute(songs, multi=True)


<generator object CMySQLCursor._execute_iter at 0x00000202EE9DB790>

In [6]:
charts ="""
CREATE TABLE IF NOT EXISTS charts (
    id INT AUTO_INCREMENT PRIMARY KEY,
    date DATE NOT NULL,
    position INT NOT NULL,
    song_id INT,
    FOREIGN KEY (song_id) REFERENCES songs(id) ON DELETE CASCADE
); """

cursor.execute(charts, multi=True)

<generator object CMySQLCursor._execute_iter at 0x00000202EE9DAF20>

In [7]:
genres = """
CREATE TABLE IF NOT EXISTS genres (
    id INT AUTO_INCREMENT PRIMARY KEY,
    year INT NOT NULL,
    genre VARCHAR(100) NOT NULL,
    frequency INT NOT NULL
); """

cursor.execute(genres, multi=True)

<generator object CMySQLCursor._execute_iter at 0x00000202EE9DB4C0>

In [8]:
cursor.close()
connection.close()